In [35]:
import pandas as pd, re, ast, gzip, shutil     # Se importan las librerías necesarias.

Con el siguiente código se extrae los datos del archivo "user_reviews.json.gz" y se guarda en un dataframe de pandas

In [10]:
data = []

with gzip.open(r"C:\Users\roylo\Downloads\PI MLOps - STEAM-20240520T190249Z-001\PI MLOps - STEAM\user_reviews.json.gz", 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

user_reviews = pd.DataFrame(data)

user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [11]:
user_reviews.info() # Se busca tener un primer vistazo a la data.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [12]:
user_url = user_reviews.pop('user_url')  # Se puede notar que la variable "user_url" no aporta información por lo que se extrae del dataframe ys e guarda en una variable.

user_reviews.head()

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In [13]:
user_reviews['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Observando la variable "reviews" se puede notar que está anidada por lo que con el siguiente código se busca crear una función que ayude a limpiarla.

In [14]:
def CleanReviews(d:list)->dict:
    """
    Ésta función extrae los valores de la lista de diccionarios, la cual es la variable original. Esto lo hace iterando, extrayendo los respectivos valores
    y añadiendolos a un nuevo diccionario con los valores en una lista. En pocas palabras, se convierte una lista de diccionarios a un solo diccionario 
    con los respectivos valores en varias listas.
    """

    dic = {"funny":[],
       "posted":[],
       "last_edited":[],
       "item_id":[],
       "helpful":[],
       "recommend":[],
       "review":[]}
    
    for i in d:
        dic["funny"].append(i.get('funny'))
        dic["posted"].append(i.get('posted'))
        dic["last_edited"].append(i.get('last_edited'))
        dic["item_id"].append(i.get("item_id"))
        dic["helpful"].append(i.get('helpful'))
        dic["recommend"].append(i.get('recommend'))
        dic["review"].append(i.get('review'))
    
    return dic

user_reviews["reviews2"] = user_reviews["reviews"].apply(CleanReviews)

reviews = user_reviews.pop('reviews') # Se extrae la variable original

user_reviews.head()

,user_id,reviews2
0,76561197970982479,"{'funny': ['', '', ''], 'posted': ['Posted Nov..."
1,js41637,"{'funny': ['', '', ''], 'posted': ['Posted Jun..."
2,evcentric,"{'funny': ['', '', '', '', '', ''], 'posted': ..."
3,doctr,"{'funny': ['', '', '', '', '', ''], 'posted': ..."
4,maplemage,"{'funny': ['3 people found this review funny',..."


In [15]:
user_reviews['reviews2'][0]

{'funny': ['', '', ''],
 'posted': ['Posted November 5, 2011.',
  'Posted July 15, 2011.',
  'Posted April 21, 2011.'],
 'last_edited': ['', '', ''],
 'item_id': ['1250', '22200', '43110'],
 'helpful': ['No ratings yet', 'No ratings yet', 'No ratings yet'],
 'recommend': [True, True, True],
 'review': ['Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.',
  "It's unique and worth a playthrough.",
  'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!']}

Continuando limpiando la varíable, se itera la variable y los valores respectivos a cada una de las llaves del diccionario se guardan en listas, creando una lista de listas. Después se crean nuevas variables con las listas obtenidas.

In [16]:
funny = []
posted = []
last_edited = []
item_id = []
helpful = []
recommend = []
review = []

for i in user_reviews['reviews2']:
    funny.append(i.get('funny'))
    posted.append(i.get('posted'))
    last_edited.append(i.get('last_edited'))
    item_id.append(i.get('item_id'))
    helpful.append(i.get('helpful'))
    recommend.append(i.get('recommend'))
    review.append(i.get('review'))

user_reviews['funny'] = funny

user_reviews['posted'] = posted

user_reviews['last_edited'] = last_edited

user_reviews['item_id'] = item_id

user_reviews['helpful']  = helpful

user_reviews['recommend'] = recommend

user_reviews['review'] = review

reviews2 = user_reviews.pop('reviews2') # Se extrae la variable y se guarda en una variable.

user_reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,"[, , ]","[Posted November 5, 2011., Posted July 15, 201...","[, , ]","[1250, 22200, 43110]","[No ratings yet, No ratings yet, No ratings yet]","[True, True, True]",[Simple yet with great replayability. In my op...
1,js41637,"[, , ]","[Posted June 24, 2014., Posted September 8, 20...","[, , ]","[251610, 227300, 239030]",[15 of 20 people (75%) found this review helpf...,"[True, True, True]",[I know what you think when you see this title...
2,evcentric,"[, , , , , ]","[Posted February 3., Posted December 4, 2015.,...","[, Last edited December 5, 2015., , , , ]","[248820, 370360, 237930, 263360, 107200, 224500]","[No ratings yet, No ratings yet, No ratings ye...","[True, True, True, True, True, True]",[A suitably punishing roguelike platformer. W...
3,doctr,"[, , , , , ]","[Posted October 14, 2013., Posted July 28, 201...","[, , , , , ]","[250320, 20920, 204100, 224600, 207610, 108710]",[2 of 2 people (100%) found this review helpfu...,"[True, True, True, True, True, True]",[This game... is so fun. The fight sequences h...
4,maplemage,"[3 people found this review funny, 1 person fo...","[Posted April 15, 2014., Posted December 23, 2...","[, , , ]","[211420, 211820, 730, 204300]",[35 of 43 people (81%) found this review helpf...,"[True, True, True, True]","[Git gud, It's like Terraria, you play for 9 h..."


Las variables ya creadas tienen varios valores dentro, esto se soluciona creando una nueva fila para cada valor. Para ésta tarea se utiliza "df.explode" pasandole las variables que queremos desglosar; y al final se resetea el índice.

In [19]:
user_reviews = user_reviews.explode(['review','recommend','helpful','last_edited','posted','funny','item_id']).reset_index(drop=True)

user_reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [20]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   funny        59305 non-null  object
 2   posted       59305 non-null  object
 3   last_edited  59305 non-null  object
 4   item_id      59305 non-null  object
 5   helpful      59305 non-null  object
 6   recommend    59305 non-null  object
 7   review       59305 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


In [21]:
# Se ve que existen valores nulos y se proceden a eliminar

user_reviews.dropna(inplace=True)
user_reviews.reset_index(drop=True,inplace=True)

user_reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [22]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   funny        59305 non-null  object
 2   posted       59305 non-null  object
 3   last_edited  59305 non-null  object
 4   item_id      59305 non-null  object
 5   helpful      59305 non-null  object
 6   recommend    59305 non-null  object
 7   review       59305 non-null  object
dtypes: object(8)
memory usage: 3.6+ MB


In [14]:
user_reviews['funny'].value_counts().to_dict()  # Se analiza si la variable "funny" aporta información. 

{'': 51154,
 '1 person found this review funny': 5180,
 '2 people found this review funny': 1232,
 '3 people found this review funny': 491,
 '4 people found this review funny': 267,
 '5 people found this review funny': 166,
 '6 people found this review funny': 103,
 '7 people found this review funny': 82,
 '8 people found this review funny': 53,
 '9 people found this review funny': 47,
 '11 people found this review funny': 38,
 '10 people found this review funny': 32,
 '12 people found this review funny': 25,
 '14 people found this review funny': 25,
 '15 people found this review funny': 24,
 '13 people found this review funny': 19,
 '20 people found this review funny': 16,
 '17 people found this review funny': 13,
 '16 people found this review funny': 12,
 '22 people found this review funny': 11,
 '27 people found this review funny': 9,
 '26 people found this review funny': 9,
 '21 people found this review funny': 9,
 '28 people found this review funny': 9,
 '25 people found this revi

In [23]:
# Puesto que la mayoría de datos de la variable están vacíos se etrae del dataframe y se guarda en una variable

funny = user_reviews.pop('funny')

user_reviews.head()

,user_id,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [24]:
user_reviews['last_edited'].value_counts().to_dict()  # Se analiza si la variable aporta información.

{'': 53165,
 'Last edited November 25, 2013.': 99,
 'Last edited October 17, 2015.': 19,
 'Last edited June 6, 2015.': 18,
 'Last edited January 3.': 17,
 'Last edited June 22, 2015.': 17,
 'Last edited September 3.': 17,
 'Last edited July 25, 2015.': 17,
 'Last edited April 28, 2015.': 16,
 'Last edited December 29, 2015.': 16,
 'Last edited April 12.': 16,
 'Last edited January 25.': 15,
 'Last edited June 24.': 15,
 'Last edited January 4, 2015.': 15,
 'Last edited January 2.': 15,
 'Last edited May 23.': 15,
 'Last edited September 21, 2015.': 15,
 'Last edited November 21, 2015.': 15,
 'Last edited April 17.': 15,
 'Last edited March 8, 2015.': 14,
 'Last edited December 16, 2015.': 14,
 'Last edited June 20, 2015.': 14,
 'Last edited October 15, 2015.': 14,
 'Last edited December 24, 2014.': 14,
 'Last edited January 11.': 14,
 'Last edited November 26, 2013.': 14,
 'Last edited November 27, 2015.': 14,
 'Last edited June 28, 2014.': 14,
 'Last edited December 8, 2015.': 13,
 'L

In [25]:
# De manera similar a la variable "funny", "last_edited" está compuesta en su mayoría por valores vacíos, por lo que se extrae del dataframe y se guarda en una variable.

last_edited = user_reviews.pop('last_edited')

user_reviews.head()

,user_id,posted,item_id,helpful,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [26]:
user_reviews['helpful'].value_counts().to_dict()  # Se analiza si la variable aporta información.

{'No ratings yet': 30168,
 '1 of 1 people (100%) found this review helpful': 6730,
 '0 of 1 people (0%) found this review helpful': 4024,
 '1 of 2 people (50%) found this review helpful': 2493,
 '2 of 2 people (100%) found this review helpful': 1872,
 '2 of 3 people (67%) found this review helpful': 1335,
 '0 of 2 people (0%) found this review helpful': 1163,
 '1 of 3 people (33%) found this review helpful': 1124,
 '3 of 3 people (100%) found this review helpful': 637,
 '3 of 4 people (75%) found this review helpful': 560,
 '2 of 4 people (50%) found this review helpful': 544,
 '1 of 4 people (25%) found this review helpful': 406,
 '0 of 3 people (0%) found this review helpful': 399,
 '2 of 5 people (40%) found this review helpful': 366,
 '3 of 5 people (60%) found this review helpful': 332,
 '4 of 4 people (100%) found this review helpful': 246,
 '4 of 5 people (80%) found this review helpful': 228,
 '3 of 6 people (50%) found this review helpful': 188,
 '0 of 4 people (0%) found this

In [27]:
# Al ver que la mayoría de la variable está compuesta por "no rating yet" se extrae la variable del dataframe y se guarda en una variable.

helpful = user_reviews.pop('helpful')

user_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


Teniendo en cuenta que la variable posted es una fecha, con el siguiente código se busca

- Primero: Limpiar la variable de palabras y puntuaciones que puedan estorbar.
    
- Segundo: Darle un formato "Datetime".

In [28]:
def CleanPosted(s:str)->str:
    if type(s) != str:
        return pd.NA
    s = s.replace('Posted','').replace(',','').replace('.','')

    s = re.sub(r"\s+"," ",s).strip()
    return s

user_reviews['posted'] = user_reviews['posted'].apply(CleanPosted)

user_reviews['posted'] = pd.to_datetime(user_reviews['posted'],format='mixed',errors='coerce')

user_reviews.dropna(inplace=True)
user_reviews.reset_index(drop=True,inplace=True)

user_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...


In [29]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49186 entries, 0 to 49185
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    49186 non-null  object        
 1   posted     49186 non-null  datetime64[ns]
 2   item_id    49186 non-null  object        
 3   recommend  49186 non-null  object        
 4   review     49186 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 1.9+ MB


In [30]:
# Se tranforma las variables "item_id" y "recommend" a formatos más descriptivos.

user_reviews['item_id'] = pd.to_numeric(user_reviews['item_id'],errors='coerce')

user_reviews['recommend'] = user_reviews['recommend'].astype(bool)

user_reviews.dropna(inplace=True)
user_reviews.reset_index(drop=True, inplace=True)

user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49186 entries, 0 to 49185
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    49186 non-null  object        
 1   posted     49186 non-null  datetime64[ns]
 2   item_id    49186 non-null  int64         
 3   recommend  49186 non-null  bool          
 4   review     49186 non-null  object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(2)
memory usage: 1.5+ MB


In [36]:
def CleanStr(s:str)->str:       # En caso de que haya valores con espacios de más.
    """
    Ésta función busca limpiar los espacios de más.
        input: Es un valor de tipo "str".

        output: Es un valor de tipo "str".
    """
    s = re.sub(r"\s+",' ',s).strip()
    return s

user_reviews['user_id'] = user_reviews['user_id'].apply(CleanStr)

user_reviews['review'] = user_reviews['review'].apply(CleanStr)

In [37]:
user_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...


In [38]:
user_reviews.to_parquet(r'C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\user_reviews.parquet',compression='snappy')

In [39]:
user_reviews = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\user_reviews.parquet")

user_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...
